In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
startTime = datetime.now()

### Open up the chrome driver that Selenium will manipulate

In [3]:
#paste the tripadvisor forum link - the below is for London
url = 'https://www.tripadvisor.ca/ShowForum-g186338-i17-London_England.html'

#for selenium set up chromedriver
wd = webdriver.Chrome('./chromedriver')
#open webpage
wd.get(url)
#maximize webpage
wd.set_window_size(1024, 600)
wd.maximize_window()

#create soup 
page_source = wd.page_source
soup = BeautifulSoup(page_source, 'html.parser')

# Set the dates you want to collect data between

### Collect Question titles and the href links for first 5 pages (100 questions)

time periods
time period 1: Nov 2020-Oct 2021
time period 2: Nov 2019-Oct 2020 (1 year ago)
time period 3: Nov 2018-Oct 2019 (2 years ago)

674 pages

In [12]:
questions = []
hrefs = []
replies = []
num = 21

#this collects the question titles and href links on the first page
#for link in soup.find_all("a", {'onclick': 'setPID(34603)'}):
#    questions.append(link.get_text().strip())
#    hrefs.append(link.get('href'))
#collecting the number of replies to each question
#for element in wd.find_elements_by_class_name('reply '):
#    replies.append(element.text)

#currently, this will click through pages num-# and collect the question titles, href links and #of replies on each page (20 per page = 180 titles)
while num < 51:
    elem = wd.find_element_by_link_text(str(num)) 
    elem.click()
    page_source = wd.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    for link in soup.find_all("a", {'onclick': 'setPID(34603)'}):
        questions.append(link.get_text().strip())
        hrefs.append(link.get('href'))
    for element in wd.find_elements_by_class_name('reply '):
        replies.append(element.text)
    time.sleep(2)
    num += 1

In [13]:
questions

['No tubes or trains to Heathrow on Boxing Day',
 'London Marathon travel advice to start line',
 'Where are the best outdoor food markets in London?',
 'Restaurants in Horley etc that do shuttle/lifts to Gatwick',
 'Covid test before flying back to London',
 'Confusing wording for Covid test rules',
 'Walk or tube?',
 'Help with a pub recom & more',
 'Heathrowe covid testing',
 'Is Tower of London strict on entry time?',
 'Westminster to Kew',
 'Bus 423 & Sheraton Hotel',
 'Travelling with 2 kids / getting around',
 'London to Salisbury',
 'most ornate and historic pubs around soho and covent garden',
 'Hotel in Angel Area',
 'The Grand/ Club Quarters - Trafalgar square',
 'Using my Visa Debit on the Underground.',
 'Farringdon pubs showing live football.',
 'Rooftop bar with 2 for 1 cocktails central London?Sundays',
 'Fielding Hotel London City Center',
 'Live Music',
 'LHR T5 to Locke at Broken Wharf',
 'UK Sim Card',
 'Gatwick airport overnight',
 'Gatwick Airport to West Hampstea

In [6]:
print(len(questions))
print(len(hrefs))
print(len(replies))

600
600
600


In [7]:
#check which indices in the list replies = 0
indices = [index for index, element in enumerate(replies) if element == '0']
indices

[39, 47, 191, 192, 210, 276, 281, 297, 301, 354, 399, 464, 521, 574]

In [8]:
for i in indices:
    print(questions[i])

Sunday afternoon live music.
Live music bar in Central London
DLR to Prince Regent station & walk to hotel - afe at night?
DLR to Prince Regent station & walk to hotel - afe at night?
Thames Clipper Tilbury/Gravesend
London Layover
London Meetup
Bookings for Ceremony of the keys open
Parking Issue (Congestion Zone)
Ceremony of the Keys
Student Visa for Spain
Anyone missing the changing
Advice for this Friday/Saturday
Art gallery weekend


In [9]:
for ele in sorted(indices, reverse = True):
    del hrefs[ele]
    del questions[ele]
    del replies[ele]

In [10]:
print(len(questions))
print(len(hrefs))
print(len(replies))

586
586
586


In [11]:
questions

['No tubes or trains to Heathrow on Boxing Day',
 'London Marathon travel advice to start line',
 'Where are the best outdoor food markets in London?',
 'Restaurants in Horley etc that do shuttle/lifts to Gatwick',
 'Covid test before flying back to London',
 'Confusing wording for Covid test rules',
 'Walk or tube?',
 'Help with a pub recom & more',
 'Heathrowe covid testing',
 'Is Tower of London strict on entry time?',
 'Westminster to Kew',
 'Bus 423 & Sheraton Hotel',
 'Travelling with 2 kids / getting around',
 'London to Salisbury',
 'most ornate and historic pubs around soho and covent garden',
 'Hotel in Angel Area',
 'The Grand/ Club Quarters - Trafalgar square',
 'Using my Visa Debit on the Underground.',
 'Farringdon pubs showing live football.',
 'Rooftop bar with 2 for 1 cocktails central London?Sundays',
 'Fielding Hotel London City Center',
 'Live Music',
 'LHR T5 to Locke at Broken Wharf',
 'UK Sim Card',
 'Gatwick airport overnight',
 'Gatwick Airport to West Hampstea

### Open a new blank tab and go to the href link for each question

In [14]:
mainQuestion1 = []
postDate1 = []
postText1 = []
postAuthor1 = []
postAuthorLocation1 = []

num = 0
wd.execute_script("window.open('about:blank', 'secondtab');")
wd.switch_to.window("secondtab")

while num < len(hrefs):
    #open up next link in list
    wd.get('https://www.tripadvisor.ca/'+hrefs[num])
    time.sleep(2)
    
    #create soup 
    page_source = wd.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    #get page numbers
    pageNums = wd.find_elements_by_class_name('paging')
    if len(pageNums) > 0:
        endNum = int(pageNums[-1].text)
    else:
        endNum = 1
    
    print('scraping ', questions[num])
    
    #collect data from page 1
    for element in wd.find_elements_by_class_name('post '):
        try:
            postAuthorLocation1.append(element.find_element_by_class_name('location').text)
            mainQuestion1.append(wd.find_element_by_css_selector('span.titleText').text)  
            postAuthor1.append(element.find_element_by_class_name('username').text)
            postDate1.append(element.find_element_by_class_name('postDate').text)
            postText1.append(element.find_element_by_class_name('postBody').text)
        except:
            pass
    
    #collect data from pages 2-endNum
    pageNum = 2

    while pageNum < endNum+1:
        try:
            elem = wd.find_element_by_link_text(str(pageNum)) 
            elem.click()
            #create soup for next page
            page_source = wd.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            time.sleep(2)

            for element in wd.find_elements_by_class_name('post '):
                try:
                    postAuthorLocation1.append(element.find_element_by_class_name('location').text)
                    mainQuestion1.append(wd.find_element_by_css_selector('span.titleText').text)
                    postAuthor1.append(element.find_element_by_class_name('username').text)
                    postDate1.append(element.find_element_by_class_name('postDate').text)
                    postText1.append(element.find_element_by_class_name('postBody').text)
                except:
                    pass
        except:
            pass
        pageNum += 1
    
    print('finished (scraping) ', questions[num], num, ' out of ', len(questions))
    num += 1
    
    
wd.close()

finished (scraping)  No tubes or trains to Heathrow on Boxing Day
scraping  London Marathon travel advice to start line
finished (scraping)  London Marathon travel advice to start line
scraping  Where are the best outdoor food markets in London?
finished (scraping)  Where are the best outdoor food markets in London?
scraping  Restaurants in Horley etc that do shuttle/lifts to Gatwick
finished (scraping)  Restaurants in Horley etc that do shuttle/lifts to Gatwick
scraping  Covid test before flying back to London
finished (scraping)  Covid test before flying back to London
scraping  Confusing wording for Covid test rules
finished (scraping)  Confusing wording for Covid test rules
scraping  Walk or tube?
finished (scraping)  Walk or tube?
scraping  Help with a pub recom & more
finished (scraping)  Help with a pub recom & more
scraping  Heathrowe covid testing
finished (scraping)  Heathrowe covid testing
scraping  Is Tower of London strict on entry time?
finished (scraping)  Is Tower of Lo

finished (scraping)  Transportation Recommendation
scraping  Hampstead Day Spa?
finished (scraping)  Hampstead Day Spa?
scraping  'Cool' restaurant recommendations?
finished (scraping)  'Cool' restaurant recommendations?
scraping  Marble Arch mound
finished (scraping)  Marble Arch mound
scraping  Quarantine hotel with at least a kitchenette/microwave
finished (scraping)  Quarantine hotel with at least a kitchenette/microwave
scraping  Greenwich Foot Tunnel day of London Marathon
finished (scraping)  Greenwich Foot Tunnel day of London Marathon
scraping  Travelling from glasgow
finished (scraping)  Travelling from glasgow
scraping  Beware of this scam
finished (scraping)  Beware of this scam
scraping  Expresstest heathrow
finished (scraping)  Expresstest heathrow
scraping  Traveling from US to London in October
finished (scraping)  Traveling from US to London in October
scraping  Restaurants near Victoria Palace Theatre
finished (scraping)  Restaurants near Victoria Palace Theatre
scrap

finished (scraping)  Travel to London from the North
scraping  Live music - cover bands
finished (scraping)  Live music - cover bands
scraping  Can I transit through UK arriving from a red list country?
finished (scraping)  Can I transit through UK arriving from a red list country?
scraping  London underground prices for 14yr and 12yrs
finished (scraping)  London underground prices for 14yr and 12yrs
scraping  Possible tube strikes in August
finished (scraping)  Possible tube strikes in August
scraping  Pub and bar reservations post-lockdown
finished (scraping)  Pub and bar reservations post-lockdown
scraping  requirement of transit or DATV
finished (scraping)  requirement of transit or DATV
scraping  Street food
finished (scraping)  Street food
scraping  VisitBritain
finished (scraping)  VisitBritain
scraping  Pre-theatre afternoon with 14 year old girls
finished (scraping)  Pre-theatre afternoon with 14 year old girls
scraping  Restaurants High Street Kensington or near
finished (scr

finished (scraping)  London day trip tomorrow
scraping  Euston to Thameslink St Pancras
finished (scraping)  Euston to Thameslink St Pancras
scraping  Wimbledon Pubs
finished (scraping)  Wimbledon Pubs
scraping  Self Connecting Flight at Heathrow
finished (scraping)  Self Connecting Flight at Heathrow
scraping  Fascinating new TV Program about the London Underground.
finished (scraping)  Fascinating new TV Program about the London Underground.
scraping  Covid question: American tourist visiting London
finished (scraping)  Covid question: American tourist visiting London
scraping  a day trip from Paris to avoid second day covid test
finished (scraping)  a day trip from Paris to avoid second day covid test
scraping  Pedestrian route to Tower Bridge
finished (scraping)  Pedestrian route to Tower Bridge
scraping  Battersea power station
finished (scraping)  Battersea power station
scraping  4 11-15 oyster cards
finished (scraping)  4 11-15 oyster cards
scraping  Portbello Market are they c

finished (scraping)  Quarantine restrictions for US citizens traveling to London
scraping  any recommended fit to fly test kit that I can take at home?
finished (scraping)  any recommended fit to fly test kit that I can take at home?
scraping  Is there any hotel near Heathrow T5 with free shuttle?
finished (scraping)  Is there any hotel near Heathrow T5 with free shuttle?
scraping  LHR: Onsite hotel or stay offsite?
finished (scraping)  LHR: Onsite hotel or stay offsite?
scraping  Play fountains
finished (scraping)  Play fountains
scraping  Covid PCR Test Question
finished (scraping)  Covid PCR Test Question
scraping  Covid Travel - US vaccinations recognition in UK?
finished (scraping)  Covid Travel - US vaccinations recognition in UK?
scraping  How long to get through checks and security at Gatwick?
finished (scraping)  How long to get through checks and security at Gatwick?
scraping  Lambeth or Canary Wharf for 2 nights?
finished (scraping)  Lambeth or Canary Wharf for 2 nights?
scr

finished (scraping)  London Underground
scraping  Pubs/bars open late on Wednesday
finished (scraping)  Pubs/bars open late on Wednesday
scraping  loading senior rail to a visitor oyster card
finished (scraping)  loading senior rail to a visitor oyster card
scraping  Reasonable pre-COVID drink prices
finished (scraping)  Reasonable pre-COVID drink prices
scraping  What's the best shopping mall in London?
finished (scraping)  What's the best shopping mall in London?
scraping  Suitable Locations Outside of London?
finished (scraping)  Suitable Locations Outside of London?
scraping  Charges for Driving
finished (scraping)  Charges for Driving
scraping  How to get tickets for an Arsenal football game?
finished (scraping)  How to get tickets for an Arsenal football game?
scraping  Visa processing time at TLScontact London
finished (scraping)  Visa processing time at TLScontact London
scraping  Parking
finished (scraping)  Parking
scraping  The Natural History Museum exhibition
finished (scr

finished (scraping)  Gluten free eating near Tower of London
scraping  I put up hotel thanks
finished (scraping)  I put up hotel thanks
scraping  Kings Cross to Waterloo, tube or train
finished (scraping)  Kings Cross to Waterloo, tube or train
scraping  activities for a 2 year old girl in London West End
finished (scraping)  activities for a 2 year old girl in London West End
scraping  Hotels outside congestion zone from M40
finished (scraping)  Hotels outside congestion zone from M40
scraping  Pre theatre meal recommendations for The Sondheim
finished (scraping)  Pre theatre meal recommendations for The Sondheim
scraping  Nice hotel bar near Covent Garden
finished (scraping)  Nice hotel bar near Covent Garden
scraping  Best Area for Couple Early Fifties (but young at heart!)
finished (scraping)  Best Area for Couple Early Fifties (but young at heart!)
scraping  Borough Market to open on Sundays
finished (scraping)  Borough Market to open on Sundays
scraping  Overnight transit
finishe

finished (scraping)  Better Time of Year, First Visit.
scraping  Best area base
finished (scraping)  Best area base
scraping  LHR to Greenwich
finished (scraping)  LHR to Greenwich
scraping  London Kings Cross to Edinburgh
finished (scraping)  London Kings Cross to Edinburgh
scraping  Dream Trip On?
finished (scraping)  Dream Trip On?
scraping  Stonehenge
finished (scraping)  Stonehenge
scraping  Art gallery weekend
finished (scraping)  Art gallery weekend
scraping  King's Cross closure
finished (scraping)  King's Cross closure
scraping  Euston to Watford Jnc trains
finished (scraping)  Euston to Watford Jnc trains
scraping  London Musical Recommendation with Kids
finished (scraping)  London Musical Recommendation with Kids
scraping  Palace of Westminster and St James’s Palace Tours
finished (scraping)  Palace of Westminster and St James’s Palace Tours
scraping  Concert at O2 but want museums,shopping too where to stay?
finished (scraping)  Concert at O2 but want museums,shopping too w

IndexError: list index out of range

In [16]:
print(len(mainQuestion1))
print(len(postDate1))
print(len(postText1))
print(len(postAuthor1))
print(len(postAuthorLocation1))

8899
8899
8899
8899
8908


In [22]:
print(mainQuestion1[5254])
print(postDate1[5254])
print(postText1[5254])
print(postAuthor1[5254])
print(postAuthorLocation1[5254])

Re: Keep your passport service for UK visa
2 years ago
Hi,
Did you get your visa? I'm wating for my decision and used the keep my passport service. I cannot find any details anywhere about this.
Thanks!
IndiraChavez
Denver, CO


In [ ]:
#dataTest = pd.DataFrame({'Post Header': mainQuestion[1812:1825],
#                    'Post Text Body':postText[1812:1825],
#                     'Post Date': postDate[1812:1825],
#                    'Author': postAuthor[1812:1825],
#                    'Author\'s Location': postAuthorLocation[1812:1825]})

#dataTest

In [ ]:
data = pd.DataFrame({'Post Header': mainQuestion,
                    'Post Text Body':postText,
                     'Post Date': postDate,
                    'Author': postAuthor,
                    'Author\'s Location': postAuthorLocation})

data

In [ ]:
data.to_csv("London_pages_21-50.csv")